In [1]:
import os
import gc
import torch
import pandas as pd
import numpy as np
import utils.constants as const

from functools import partial

from utils.losses import MosLoss
from utils.xception import Mos_Xception
from utils.data_loader import get_data_loaders
from utils.metrics import accuracy, macro_f1

from fastai.vision.all import *
from fastai.distributed import setup_distrib, num_distrib

/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
device = torch.device('cuda:0')

In [3]:
df = pd.read_csv('data/splits/data_fold_1.csv')

In [4]:
train_dl, val_dl = get_data_loaders(df)

In [5]:
dls = DataLoaders(train_dl, val_dl)

In [6]:
loss = MosLoss(const.NUM_CLASSES)
freeze_bn = False
save_imgs = False
train_losses = []
valid_losses = []
valid_f1s = []
lr_hist = []
metrics = [accuracy, macro_f1]

Focal Loss with gamma =  0


In [7]:
result_dict = {}

for idx in range(40):
    mom = np.random.uniform(0.5, 0.999)
    alpha = np.random.uniform(0.5, 0.999)
    eps = np.random.uniform(1e-6, 0.1)
    wd = np.random.uniform(1e-6, 0.1)

    opt_func = partial(ranger, mom=mom, alpha=alpha, eps=eps)
    net = Mos_Xception(const.NUM_CLASSES)

    learn = Learner(
        dls,
        net,
        wd=wd,
        opt_func=opt_func,
        metrics=metrics,
        loss_func=loss,
    )

    learn.fit_one_cycle(
        20,
        2e-03,
        div=25,
        pct_start=0.3,
    )

    param_dict = {}
    param_dict['mom'] = mom
    param_dict['alpha'] = alpha
    param_dict['eps'] = eps
    param_dict['wd'] = wd
    param_dict['f1'] = learn.metrics[1].value.detach().cpu().numpy()

    result_dict[idx] = param_dict

    del net
    del learn
    gc.collect()
    torch.cuda.empty_cache()

epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.388383,4.626642,0.105769,0.031825,00:48
1,3.753207,3.094482,0.371795,0.207645,00:43
2,3.074545,2.640740,0.560897,0.427188,00:41
3,2.270517,1.555839,0.746795,0.547961,00:40
4,1.527138,0.945967,0.798077,0.604068,00:40
5,1.099044,0.671763,0.894231,0.705602,00:40
6,0.851959,0.614067,0.881410,0.749498,00:39
7,0.668011,0.603627,0.871795,0.748958,00:39
8,0.543086,0.466125,0.907051,0.727374,00:41
9,0.467613,0.334209,0.945513,0.840258,00:39


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.494915,4.695302,0.134615,0.094527,00:39
1,3.763239,3.057135,0.467949,0.313587,00:39
2,2.956151,2.381108,0.634615,0.492284,00:40
3,2.058707,1.312881,0.759615,0.624878,00:39
4,1.340815,0.753065,0.881410,0.720202,00:39
5,0.998069,0.763396,0.833333,0.700978,00:40
6,0.779395,0.524658,0.891026,0.777396,00:40
7,0.636154,0.418240,0.916667,0.759758,00:39
8,0.546426,0.324701,0.932692,0.786983,00:40
9,0.455163,0.376931,0.916667,0.799521,00:41


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.570750,4.837300,0.176282,0.054570,00:40
1,3.874661,3.150304,0.285256,0.146938,00:40
2,3.151832,2.830421,0.490385,0.371906,00:42
3,2.570969,1.937144,0.612179,0.528033,00:50
4,1.907531,1.357117,0.766026,0.589991,00:59
5,1.411213,0.989358,0.830128,0.655266,00:53
6,1.091150,0.944586,0.791667,0.617325,00:53
7,0.930081,0.821382,0.810897,0.680903,00:46
8,0.746845,0.646447,0.858974,0.732689,00:47
9,0.673956,0.555879,0.891026,0.700743,00:43


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.270496,4.350190,0.217949,0.073766,00:53
1,3.625669,2.999589,0.458333,0.251232,00:50
2,2.864018,2.141290,0.698718,0.557424,00:54
3,1.936285,1.152844,0.817308,0.600003,01:04
4,1.281200,0.757415,0.849359,0.708487,01:11
5,0.939317,0.744052,0.846154,0.707452,01:03
6,0.701419,0.624985,0.868590,0.701612,00:47
7,0.583972,0.322344,0.951923,0.817118,00:47
8,0.514692,0.562695,0.887821,0.733011,00:46
9,0.411940,0.525238,0.903846,0.728181,00:46


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.326081,4.492260,0.108974,0.038997,00:46
1,3.695799,3.088612,0.266026,0.204560,00:45
2,3.034040,2.586499,0.528846,0.458117,00:48
3,2.254935,1.622450,0.679487,0.538050,00:47
4,1.545960,0.960519,0.846154,0.660224,00:46
5,1.132451,0.599189,0.913462,0.716815,00:45
6,0.825319,0.624347,0.881410,0.691333,00:45
7,0.681744,0.550427,0.897436,0.735889,00:45
8,0.601090,0.365117,0.945513,0.835752,00:40
9,0.510621,0.319350,0.932692,0.815163,00:38


...

epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.480604,4.740539,0.105769,0.063298,00:54
1,3.800413,3.090811,0.381410,0.185399,00:56
2,3.087356,2.638223,0.580128,0.419961,00:54
3,2.289302,1.595000,0.660256,0.569058,00:53
4,1.600831,1.019678,0.791667,0.642905,00:52
5,1.167267,0.717864,0.865385,0.751488,00:53
6,0.882510,0.610941,0.875000,0.706219,00:51
7,0.732744,0.714207,0.839744,0.738768,00:51
8,0.602704,0.450563,0.923077,0.786120,00:54
9,0.470917,0.295759,0.948718,0.825680,00:52


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.546942,4.711398,0.144231,0.096610,00:54
1,3.827469,3.106480,0.352564,0.183789,00:56
2,3.115403,2.752779,0.576923,0.454597,00:57
3,2.461554,1.844105,0.596154,0.506556,00:57
4,1.789437,1.221277,0.766026,0.607979,00:57
5,1.291622,0.897156,0.817308,0.692792,00:58
6,0.972619,0.808600,0.842949,0.727167,00:57
7,0.751177,0.590295,0.878205,0.681364,00:59
8,0.718023,0.495045,0.919872,0.755756,00:59
9,0.555606,0.459787,0.916667,0.773794,00:59


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.118750,4.208874,0.108974,0.064229,00:58
1,3.525280,2.965911,0.458333,0.341873,00:56
2,2.624452,1.713090,0.714744,0.522537,00:54
3,1.638039,0.989586,0.807692,0.678728,00:57
4,1.095752,0.891355,0.798077,0.607472,00:56
5,0.800682,0.486683,0.913462,0.789743,00:56
6,0.702318,0.554834,0.881410,0.719515,00:57
7,0.568287,0.432162,0.900641,0.779693,00:56
8,0.435289,0.317327,0.948718,0.826632,00:55
9,0.434840,0.347850,0.935897,0.850066,00:56


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.551510,4.849012,0.096154,0.040976,00:52
1,3.905227,3.159143,0.269231,0.113293,00:51
2,3.188901,2.876618,0.592949,0.422355,00:48
3,2.618545,2.004645,0.660256,0.507553,00:52
4,1.915954,1.311949,0.733974,0.561425,00:51
5,1.380123,0.910552,0.830128,0.729676,00:48
6,1.023581,0.788143,0.823718,0.682446,00:52
7,0.848860,0.635282,0.887821,0.729995,00:50
8,0.692253,0.603904,0.891026,0.716635,00:48
9,0.562774,0.525140,0.897436,0.808524,00:51


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,5.754382,5.229441,0.121795,0.045857,00:49
1,4.140485,3.298325,0.246795,0.102432,00:51
2,3.285674,2.971600,0.464744,0.311375,00:50
3,2.817011,2.348865,0.612179,0.510016,00:48
4,2.167916,1.636243,0.727564,0.561180,00:51
5,1.644097,1.157609,0.826923,0.676408,00:50
6,1.281562,0.873011,0.839744,0.701778,00:48
7,1.014905,0.684178,0.884615,0.738608,00:53
8,0.827829,0.645408,0.871795,0.723135,00:49
9,0.681789,0.494928,0.923077,0.809784,00:49


In [8]:
f1_list = []

for v in result_dict.values():
    f1_list.append(v['f1'])

In [9]:
best_idx = np.argmax(f1_list)

In [10]:
f1_list[best_idx]

array(0.95580472)

In [11]:
result_dict[best_idx]

{'mom': 0.9531249799613819,
 'alpha': 0.7216872276263672,
 'eps': 0.02241164469571523,
 'wd': 0.04258532691804346,
 'f1': array(0.95580472)}

In [12]:
with open('param.txt', 'w') as f:
    for k, v in result_dict[best_idx].items():
        f.write(f'{k}: {v}\n')